## Import library and dataset

In [11]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.metrics import precision_score
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, \
    EarlyStoppingCallback
import pandas as pd

dataset = pd.read_csv(f"../dataset/preprocess/github-labels-top3-803k-1.0%.csv")
print(dataset.issue_label.value_counts())
print(dataset.info())

issue_label
0.0    4014
1.0    3326
2.0     695
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8035 entries, 0 to 8034
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0.2              8035 non-null   int64  
 1   Unnamed: 0.1              8035 non-null   int64  
 2   Unnamed: 0                8035 non-null   int64  
 3   issue_url                 8035 non-null   object 
 4   issue_label               8035 non-null   float64
 5   issue_created_at          8035 non-null   object 
 6   issue_author_association  8035 non-null   object 
 7   repository_url            8035 non-null   object 
 8   issue_title               8035 non-null   object 
 9   issue_body                7242 non-null   object 
 10  text                      8035 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 690.6+ KB
None


## preprocess the dataset
### process the data

In [12]:
dataset = dataset.rename(columns={"issue_label": "label"})
# label required to be int64, otherwise will get error during training
dataset['label'] = dataset['label'].astype('int64')

pd.set_option('future.no_silent_downcasting', True)

### tokenize the data

In [13]:
tokenizer = AutoTokenizer.from_pretrained("funnel-transformer/small-base")

encodings = tokenizer(
    dataset.text.values.tolist(),
    max_length = 128,
    truncation = True,
    padding="max_length",
    return_attention_mask=True,
    return_tensors='pt'
)

print(encodings)

# dataset = Dataset.from_pandas(df)
# print(dataset)
# def preprocess(batch):
#     return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)
#
# dataset = dataset.map(preprocess, batched=True)
# dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

{'input_ids': tensor([[  101,  3444,  5227,  ...,     0,     0,     0],
        [  101,  1996,  2944,  ...,     0,     0,     0],
        [  101, 12441,  7361,  ...,     0,     0,     0],
        ...,
        [  101,  3114,  2121,  ...,  4464,  2213,   102],
        [  101,  2622,  3030,  ...,     0,     0,     0],
        [  101,  2025,  5026,  ...,  1996,  2956,   102]]), 'token_type_ids': tensor([[2, 0, 0,  ..., 0, 0, 0],
        [2, 0, 0,  ..., 0, 0, 0],
        [2, 0, 0,  ..., 0, 0, 0],
        ...,
        [2, 0, 0,  ..., 0, 0, 0],
        [2, 0, 0,  ..., 0, 0, 0],
        [2, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}


### split the data

In [14]:
trainer_dataset = Dataset.from_dict({
    **encodings,
    "label": dataset.label.values
})

trainer_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print(trainer_dataset)

# how to do stratify
split_data = trainer_dataset.train_test_split(test_size=0.15)
train_set = split_data['train']
test_set = split_data['test']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 8035
})


In [15]:
model = AutoModelForSequenceClassification.from_pretrained("funnel-transformer/small-base", num_labels=3)

# Define training args
training_args = TrainingArguments(
    output_dir="../dataset",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    num_train_epochs=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "precision": precision_score(labels, predictions, average="weighted"),
        "recall": recall_score(labels, predictions, average="weighted"),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

# save model
# trainer.save_model("trained_electra_model")

# save tokenizer
# tokenizer.save_pretrained("")


Some weights of FunnelForSequenceClassification were not initialized from the model checkpoint at funnel-transformer/small-base and are newly initialized: ['classifier.linear_hidden.bias', 'classifier.linear_hidden.weight', 'classifier.linear_out.bias', 'classifier.linear_out.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/funnel/modeling_funnel.py:336: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor in

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.902200,0.778041,0.733831,0.674787,0.733831,0.698683


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/funnel/modeling_funnel.py:336: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which wil

KeyboardInterrupt: 